In [61]:
# pip install Pillow
# pip install selenium
# pip install requests

In [62]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
import requests
import io
from PIL import Image as Img
import time

In [63]:
PATH = "chromedriver.exe"

In [64]:
wd = wd.Chrome(PATH)

C:\Users\admin\AppData\Local\Temp\ipykernel_11952\1449107096.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = wd.Chrome(PATH)


In [ ]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?q=Capacitors&tbm=isch&ved=2ahUKEwiLpMvS98_-AhUfmycCHWpGAEEQ2-cCegQIABAA&oq=Capacitors&gs_lcp=CgNpbWcQAzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIFCAAQgAQyBwgAEIoFEEMyBwgAEIoFEEMyBwgAEIoFEEMyBwgAEIoFEEMyBQgAEIAEUABYAGDNGmgBcAB4AIAB1QWIAdUFkgEDNi0xmAEAqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=r35NZMuQJJ-2nsEP6oyBiAQ&bih=696&biw=1536"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.pT0Scc")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.r48jcc')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".iPVvYb")
            if load_more_button:
                wd.execute_script("document.querySelector('.iPVvYb').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [65]:
# def get_images_from_google(wd, delay, max_images):
#     def scroll_down(wd):
#         wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(delay)
        
#     url = "https://www.google.com/search?q=Capacitors&tbm=isch&ved=2ahUKEwiLpMvS98_-AhUfmycCHWpGAEEQ2-cCegQIABAA&oq=Capacitors&gs_lcp=CgNpbWcQAzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIFCAAQgAQyBwgAEIoFEEMyBwgAEIoFEEMyBwgAEIoFEEMyBwgAEIoFEEMyBQgAEIAEUABYAGDNGmgBcAB4AIAB1QWIAdUFkgEDNi0xmAEAqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=r35NZMuQJJ-2nsEP6oyBiAQ&bih=696&biw=1536"
#     wd.get(url)
    
#     image_urls = set()
    
#     while len(image_urls) < max_images:
#         scroll_down(wd)
        
#         thumbnails = wd.find_elements(By.CLASS_NAME, "pT0Scc")
        
#         for img in thumbnails[len(image_urls)+skips : max_images]:
#             try:
#                 img.click()
#                 time.sleep(delay)
#             except Exception:
#                 continue
                
#             images = wd.find_elements(By.CLASS_NAME, "r48jcc")
#             for image in images:
#                 if image.get_attribute('src') in image_urls:
#                     max_image +=1
#                     skips += 1
#                     break
                
                
#                 if image.get_attribute('src') and 'http' in image.get.attribute('src'):
#                     image_urls.add(image.get_attribute('src'))
#                     print(f"Found {len(image_urls)}")
                    
    
#     return image_urls

In [ ]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [66]:
# def download_images(download_path, url, file_name):
#     try:
#         image_content = requests.get(url).content
#         image_file = io.BytesIO(image_content)
#         image = Img.open(image_file)
#         file_path = download_path

#         with open(file_path, "wb") as f:
#             image.save(f, "JPEG")

#         print("Success")
#     except Exception as e:
#         print("FAILED -", e)
        

In [ ]:
def search_and_download(search_term:str,driver_path:str,target_path='',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [68]:
search_term = 'capacitor'

search_and_download(search_term=search_term, driver_path=PATH)

KeyboardInterrupt: 